# build simple LLM app using LCEL


In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")

In [7]:
from langchain_groq import ChatGroq
groq_llm = ChatGroq(
    model_name="gemma2-9b-it",
    api_key=groq_api_key,
    temperature=0.5,
    max_tokens=1000,
    #top_p=0.9,
    #top_k=40,
    max_retries=3,
    timeout=10,
    #max_completion_tokens=1000
)

# Str output parser chain with LCEL

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following text to French"),
    HumanMessage(content="I love programming in Python")
]

response = groq_llm.invoke(messages)

In [10]:
response

AIMessage(content="J'adore programmer en Python. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 20, 'total_tokens': 31, 'completion_time': 0.02, 'prompt_time': 0.00135974, 'queue_time': 0.088471442, 'total_time': 0.02135974}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1b189467-2315-4b94-9e85-c6ee501adbc9-0', usage_metadata={'input_tokens': 20, 'output_tokens': 11, 'total_tokens': 31})

In [11]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(response)

"J'adore programmer en Python. \n"

In [12]:
## using LCEL we can chian the components together
chain = groq_llm | parser
chain.invoke(messages)

"J'adore programmer en Python. \n"

In [13]:
## prompt templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following text to {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user", "{text}") 
    ]
)

In [16]:
result = prompt_template.invoke({"language": "French", "text": "I love programming in Python"})


In [17]:
result.to_messages()

[SystemMessage(content='Translate the following text to French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love programming in Python', additional_kwargs={}, response_metadata={})]

In [19]:
chain = prompt_template | groq_llm | parser
chain.invoke({"language":"French", "text":"I love programming in Python"})


"J'adore programmer en Python. \n"

# LANGSERVE

# FOR DEPLOYING RESTAPI

In [ ]:
import 